In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('Indian_Hotels_data.pdf')
docs=loader.load()


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents[:5]

[Document(metadata={'producer': '', 'creator': 'WPS Spreadsheets', 'creationdate': '2025-04-30T18:04:54+05:30', 'author': '', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2025-04-30T18:07:58+06:30', 'sourcemodified': "D:20250430180454+05'30'", 'subject': '', 'title': '', 'trapped': '/False', 'source': 'Indian_Hotels_data.pdf', 'total_pages': 69, 'page': 0, 'page_label': '1'}, page_content='Hotel_Name Hotel_RatinCity Star_Rating Breakfast Wifi\nCrowne Plaza Kochi 4.6 kochi 5-star hotel Free breakfaFree Wi-Fi\nTrident Hotel Cochin 4.5 kochi 5-star hotel Free breakfaWi-Fi\nThe Galaxy Suites 3.8 kochi Apartment Sleeps 10 Free parking\nThe Renai cochin 4.2 kochi 4-star hotel Free breakfaFree Wi-Fi\nRamada by Wyndham Kochi 4.5 kochi 5-star hotel Breakfast Free Wi-Fi\nThe Renai cochin 4.2 kochi 4-star hotel Free breakfaFree Wi-Fi\nRadisson Blu Hotel, Kochi 4.3 kochi 5-star hotel Breakfast Free Wi-Fi\nHoliday Inn Cochin, an IHG Hotel 4.4 kochi 5-star hotel Breakfast Free Wi-Fi\nO

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

In [4]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db = Chroma.from_documents(documents, embeddings)

C:\Users\sagar\AppData\Local\Temp\ipykernel_4388\2478136966.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [5]:
query = "Show me best hotels in goa"
retireved_results=db.similarity_search(query)
print(retireved_results[0].page_content)

Radisson Resort Goa Baga 4.8 goa 4-star hotel Breakfast Free Wi-Fi
W Goa 4.5 goa 5-star hotel Free breakfaFree Wi-Fi
The Hosteller Goa, Arpora 4.3 goa Breakfast Free Wi-Fi Free parking
Grand Hyatt Goa 4.6 goa 5-star hotel Breakfast Free Wi-Fi
Little India Beach Cottages 4 goa House Sleeps 10 5 bedrooms
Holiday Inn Resort Goa, an IHG Hotel 4.6 goa 5-star hotel Breakfast Free Wi-Fi
Agonda Paradise Beach Huts 4.2 goa Sleeps 9 3 bedrooms Air condition
Pitruchaya Sea View Cottages by Vyom T 4.1 goa 0 0 0
The LaLiT Golf & Spa Resort Goa 4.4 goa 5-star hotel Free breakfaWi-Fi
Whoopers Boutique Hotel, Anjuna 4.9 goa 0 0 0
The Hosteller Goa, Candolim 4.7 goa Free Wi-Fi Free parkingPool
Tropical Bay 3.8 goa Free breakfaFree Wi-Fi Free parking
Cola Goa Beach Resort 4.3 goa Free breakfaFree Wi-Fi Free parking
Blue Lagoon Resort | Room & Restauran 3.9 goa Villa Sleeps 8 3 bedrooms
Radisson Blu Resort Goa Cavelossim Bea 4.4 goa 5-star hotel Breakfast Free Wi-Fi


In [6]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3") 

C:\Users\sagar\AppData\Local\Temp\ipykernel_4388\3282379104.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(""" 
Answere the following questionn based only on the provided context.
Think step by step before providing a detailed annswer.
<context>
{context}
</context>
Question: {input}""")

In [13]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain =create_stuff_documents_chain(llm,prompt)

In [14]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002B7BE0EBA30>, search_kwargs={})

In [15]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [16]:
response=retrieval_chain.invoke({"input":"Show me best hotels in pune"})

In [17]:
response['answer']

'Based on the provided context, here are some of the best hotels in Pune:\n\n**5-Star Hotels**\n\n1. Sheraton Grand Pune Bund Garden Hotel (4.5/5)\n2. Four Points by Sheraton Hotel & Service (4.3/5)\n3. Royal Orchid Central Pune (4.3/5)\n\n**4-Star Hotels**\n\n1. The Oakwood (4.0/5)\n2. Pratikruti The Lakeview Resort (4.2/5)\n3. Fairfield by Marriott Pune Kharadi (4.3/5)\n4. Hotel Centro (4.4/5)\n5. StayBird - B Suite, Business Hotel (4.0/5)\n\nThese hotels are highly rated for their amenities, service, and overall experience.'